In [3]:
import geopandas as gpd
import ipywidgets as widgets
import pandas as pd
from lonboard import Map, PathLayer


# Streets

We'll use DuckDB to query the the Overture Transport data to Greater London extract of roads.

DuckDB is an analytics tool that allows you to query remote files and download only the data you want. You can follow the installation steps [here](https://duckdb.org/docs/installation/).

DuckDB integrates great with Parquet files; some good use cases can be found:
1. [DuckDB: The indispensable geospatial tool you didn’t know you were missing](https://medium.com/radiant-earth-insights/duckdb-the-indispensable-geospatial-tool-you-didnt-know-you-were-missing-5fe11c5633e5?) (Chris Holmes) - good introduction to using DuckDB with GeoParquet for geospatial analytical purposes, covers everything from usability, performance, geospatial support.
2. [Admin-partitioned GeoParquet distribution](https://medium.com/radiant-earth-insights/the-admin-partitioned-geoparquet-distribution-59f0ca1c6d96?) (Chris Holmes) - more complex use case looking at making use of parquet hive paritioning working on the Google Buildings data.
3. [My First Billion (of Rows) in DuckDB](https://towardsdatascience.com/my-first-billion-of-rows-in-duckdb-11873e5edbb5?) (João Pedro) - non-spatial example.

First - we'll get rough approximate administration boundaries for Greater London from ONS.

In [13]:
london_gdf = gpd.read_file(
    "https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Local_Authority_Districts_May_2024_Boundaries_UK_BUC/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson",
    use_arrow=True,
    columns=["LAD24CD", "LAD24NM", "geometry"],
    where="LAD24CD LIKE 'E09%'"
)

tuple(london_gdf.total_bounds)

(-0.509807321692741, 51.2867904040814, 0.334024218329939, 51.6918756451179)

We can then make a DuckDB command to extract the data for our bounding box:

```sql
            LOAD spatial;
            LOAD httpfs;
            SET s3_region='us-west-2';

            COPY (
            SELECT
                *
            FROM read_parquet('s3://overturemaps-us-west-2/release/2024-08-20.0/theme=transportation/type=segment/*')
            WHERE
                subtype = 'subtype' AND
                bbox.xmin > 0.509807321692741 AND bbox.xmax < 0.334024218329939 AND
                bbox.ymin > 51.2867904040814 AND bbox.ymax < 51.6918756451179
            )
            TO 'data/london_roads.parquet';

```

Should save a Parquet file ~120Mb.

In [8]:
roads_df = pd.read_parquet("data/london_roads.parquet")

roads_gdf = gpd.GeoDataFrame(
    roads_df, geometry=gpd.GeoSeries.from_wkb(roads_df.geometry, crs=4326)
)

len(roads_gdf)

534471

In [9]:
poly_mask = london_gdf.union_all(method="coverage")

roads_gdf = roads_gdf.iloc[
    roads_gdf.sindex.query(
        poly_mask,
        predicate="intersects",
    )
]

len(roads_gdf)

427423

Slimmed data own from 534K features to 427K.

In [10]:
layer = PathLayer.from_geopandas(
    roads_gdf[["subtype", "class", "geometry"]], width_min_pixels=0.8
)

In [11]:
view_state = {
    "longitude": poly_mask.centroid.x,
    "latitude": poly_mask.centroid.y,
    "zoom": 9,
    "pitch": 0,
    "bearing": 0,
}

map_layout = widgets.Layout(height="1200px")

m = Map(layer, view_state=view_state, layout=map_layout)

m

Map(layers=[PathLayer(table=pyarrow.Table
subtype: string
class: string
__index_level_0__: int64
geometry: lis…

In [12]:
m.to_html("examples/streets.html")